In [1]:
!pip install stanza -q

DEPRECATION: Loading egg at c:\users\vrishank\appdata\local\programs\python\python312\lib\site-packages\object_detection-0.1-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\vrishank\appdata\local\programs\python\python312\lib\site-packages\pyparsing-2.4.7-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\vrishank\appdata\local\programs\python\python312\lib\site-packages\sacrebleu-2.2.0-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330

[notice] A new release of pi

In [1]:
import stanza
from nltk import Tree

stanza.download('en')
nlp = stanza.Pipeline(lang='en', processors='tokenize,pos,constituency')


C:\Users\Vrishank\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\utils\_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


2025-06-24 13:20:55 INFO: Downloaded file to C:\Users\Vrishank\stanza_resources\resources.json
2025-06-24 13:20:55 INFO: Downloading default packages for language: en (English) ...
2025-06-24 13:20:56 INFO: File exists: C:\Users\Vrishank\stanza_resources\en\default.zip
2025-06-24 13:21:01 INFO: Finished downloading models and saved to C:\Users\Vrishank\stanza_resources
2025-06-24 13:21:01 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2025-06-24 13:21:01 INFO: Downloaded file to C:\Users\Vrishank\stanza_resources\resources.json
2025-06-24 13:21:01 WARNING: Language en package default expects mwt, which has been added
2025-06-24 13:21:02 INFO: Loading these models for language: en (English):
| Processor    | Package             |
--------------------------------------
| tokenize     | combined            |
| mwt          | combined            |
| pos          | combined_charlm     |
| constituency | ptb3-revised_charlm |

2025-06-24 13:21:02 INFO: Using device: cpu
2025-06-24 13:21:02 INFO: Loading: tokenize
2025-06-24 13:21:05 INFO: Loading: mwt
2025-06-24 13:21:05 INFO: Loading: pos
2025-06-24 13:21:07 INFO: Loading: constituency
2025-06-24 13:21:08 INFO: Done loading processors!


In [2]:
doc = nlp("delete files with inode number specified by REGEX under current directory")
tree = Tree.fromstring(str(doc.sentences[0].constituency))
tree.pretty_print()

        ROOT                                                                          
         |                                                                             
         S                                                                            
         |                                                                             
         VP                                                                           
   ______|____                                                                         
  |           NP                                                                      
  |       ____|_______________                                                         
  |      |                    PP                                                      
  |      |     _______________|_______________                                         
  |      |    |                               NP                                      
  |      |    |           ____________

In [11]:
def get_node_phrases(tree, depth = 2):
    leaf_positions = tree.treepositions('leaves')
    word_to_phrase = []

    for pos in leaf_positions:
        try:
            # Go up two levels from the leaf
            parent_1 = pos[:-1]       # immediate parent
            parent_2 = pos[:-2]
            if depth == 2: # two levels up
                phrase_subtree = tree[parent_2]
            elif depth == 1: 
                phrase_subtree = tree[parent_1]
            phrase = " ".join(phrase_subtree.leaves())
            word = tree[pos]
            word_to_phrase.append((word, phrase, phrase_subtree.label()))
        except IndexError:
            # Skip if not enough levels
            continue

    return word_to_phrase

phrases_data = get_node_phrases(tree)

for word, phrase, label in phrases_data:
    print(f"Word: '{word}' → Phrase: '{phrase}' (Type: {label})")


Word: 'delete' → Phrase: 'delete files with inode number specified by REGEX under current directory' (Type: VP)
Word: 'files' → Phrase: 'files' (Type: NP)
Word: 'with' → Phrase: 'with inode number specified by REGEX under current directory' (Type: PP)
Word: 'inode' → Phrase: 'inode number' (Type: NP)
Word: 'number' → Phrase: 'inode number' (Type: NP)
Word: 'specified' → Phrase: 'specified by REGEX under current directory' (Type: VP)
Word: 'by' → Phrase: 'by REGEX' (Type: PP)
Word: 'REGEX' → Phrase: 'REGEX' (Type: NP)
Word: 'under' → Phrase: 'under current directory' (Type: PP)
Word: 'current' → Phrase: 'current directory' (Type: NP)
Word: 'directory' → Phrase: 'current directory' (Type: NP)


In [12]:
phrases = []
for word, phrase, label in phrases_data:
    phrases.append(phrase)

phrases

['delete files with inode number specified by REGEX under current directory',
 'files',
 'with inode number specified by REGEX under current directory',
 'inode number',
 'inode number',
 'specified by REGEX under current directory',
 'by REGEX',
 'REGEX',
 'under current directory',
 'current directory',
 'current directory']

Now that we have that,
* remove the phrases with more than 4 words. 
* add the missing words in the right order. ("delete" is missing here)
* remove duplicate phrases
* remove redundant phrases ("by REGEX" and "REGEX", "current directory" and "under current directory")
* 

In [13]:
# remove phrases with more than 4 words
for i, phrase in enumerate(phrases):
    if len(phrase.split()) > 4:
        del phrases[i]

phrases

['files',
 'inode number',
 'inode number',
 'by REGEX',
 'REGEX',
 'under current directory',
 'current directory',
 'current directory']

In [14]:
# remove duplicate phrases
for i,phrase1 in enumerate(phrases):
    for j,phrase2 in enumerate(phrases):
        if j <= i:
            continue

        if phrase1 == phrase2:
            del phrases[j]

phrases

['files',
 'inode number',
 'by REGEX',
 'REGEX',
 'under current directory',
 'current directory']

whats left:
* remove redundant phrases ("by REGEX" and "REGEX", "current directory" and "under current directory")
* add the missing words in the right order. ("delete" is missing here)
* 

In [18]:
# remove redundant phrases
phrases = [
    p for p in phrases
    if not any(p != q and p in q for q in phrases)
]
phrases

['files', 'inode number', 'by REGEX', 'under current directory']

In [16]:
# add missing words in the right order

words = []
words_data = get_node_phrases(tree, depth=1)
for word, phrase, label in words_data:
    print(f"Word: '{word}' → Phrase: '{phrase}' (Type: {label})")
    words.append(phrase)


Word: 'delete' → Phrase: 'delete' (Type: VB)
Word: 'files' → Phrase: 'files' (Type: NNS)
Word: 'with' → Phrase: 'with' (Type: IN)
Word: 'inode' → Phrase: 'inode' (Type: NN)
Word: 'number' → Phrase: 'number' (Type: NN)
Word: 'specified' → Phrase: 'specified' (Type: VBN)
Word: 'by' → Phrase: 'by' (Type: IN)
Word: 'REGEX' → Phrase: 'REGEX' (Type: NNP)
Word: 'under' → Phrase: 'under' (Type: IN)
Word: 'current' → Phrase: 'current' (Type: JJ)
Word: 'directory' → Phrase: 'directory' (Type: NN)


In [17]:
words

['delete',
 'files',
 'with',
 'inode',
 'number',
 'specified',
 'by',
 'REGEX',
 'under',
 'current',
 'directory']

In [19]:
phrase_tokens = {tuple(p.split()): p for p in phrases}

result = []
i = 0
while i < len(words):
    for length in range(len(words), 0, -1):
        slice_ = tuple(words[i:i+length])
        if slice_ in phrase_tokens:
            result.append(phrase_tokens[slice_])
            i += length
            break
    else:
        result.append(words[i])
        i += 1

In [20]:
result

['delete',
 'files',
 'with',
 'inode number',
 'specified',
 'by REGEX',
 'under current directory']